# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 8:37AM,244127,10,8395115,Beach Products Inc,Released
1,Aug 15 2022 8:37AM,244127,10,8404224,Beach Products Inc,Released
2,Aug 15 2022 8:37AM,244127,10,8404225,Beach Products Inc,Released
3,Aug 15 2022 8:37AM,244127,10,8404226,Beach Products Inc,Released
4,Aug 15 2022 8:37AM,244127,10,8404227,Beach Products Inc,Released
5,Aug 15 2022 8:37AM,244127,10,8404228,Beach Products Inc,Released
6,Aug 15 2022 8:37AM,244127,10,8404229,Beach Products Inc,Released
7,Aug 15 2022 8:37AM,244127,10,8404230,Beach Products Inc,Released
8,Aug 15 2022 8:37AM,244127,10,8404231,Beach Products Inc,Released
9,Aug 15 2022 8:37AM,244127,10,8404232,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,244121,Released,3
11,244122,Released,6
12,244123,Released,8
13,244126,Released,1
14,244127,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244121,NaN,3.0
244122,NaN,6.0
244123,NaN,8.0
244126,NaN,1.0
244127,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244107,0.0,10.0
244108,0.0,11.0
244109,0.0,27.0
244110,0.0,15.0
244111,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244107,0,10
244108,0,11
244109,0,27
244110,0,15
244111,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244107,0,10
1,244108,0,11
2,244109,0,27
3,244110,0,15
4,244111,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244107,,10
1,244108,,11
2,244109,,27
3,244110,,15
4,244111,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc
13,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.
19,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc."
20,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc."
30,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation
41,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation
68,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation
83,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health
84,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC"
92,Aug 15 2022 8:12AM,244121,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,,13
1,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,,6
2,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",,1
3,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",,10
4,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,,11
5,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,,27
6,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,,15
7,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,,1
8,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC",,8
9,Aug 15 2022 8:12AM,244121,10,Emerginnova,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,
1,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6,
2,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1,
3,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,
4,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,
5,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27,
6,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,15,
7,Aug 15 2022 8:30AM,244111,10,Ivaoes Animal Health,1,
8,Aug 15 2022 8:22AM,244123,10,"Snap Medical Industries, LLC",8,
9,Aug 15 2022 8:12AM,244121,10,Emerginnova,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,
1,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6,
2,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1,
3,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,
4,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,NaN
1,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6,NaN
2,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1,NaN
3,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,NaN
4,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,0.0
1,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6,0.0
2,Aug 15 2022 8:32AM,244126,19,"GUSA Granules USA, Inc.",1,0.0
3,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,0.0
4,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,11,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1952916,88,0.0
12,488237,36,0.0
19,488242,27,2.0
20,488242,8,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1952916,88,0.0
1,12,488237,36,0.0
2,19,488242,27,2.0
3,20,488242,8,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,88,0.0
1,12,36,0.0
2,19,27,2.0
3,20,8,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,12,Released,36.0
2,19,Released,27.0
3,20,Released,8.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,0.0,0.0,2.0,0.0
Released,88.0,36.0,27.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,0.0,0.0,2.0,0.0
1,Released,88.0,36.0,27.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,0.0,0.0,2.0,0.0
1,Released,88.0,36.0,27.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()